<a href="https://colab.research.google.com/github/sruthi1996/DeepLearning/blob/master/p1ch5_optimizerswithPyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch
torch.set_printoptions(edgeitems=2)
import torch.optim as optim

In [0]:
t_c = torch.tensor([0.5, 10.0, 15.0, 40.0, 10.0, 10.0, 3.0, -4.0, 6.0, 13.0, -2.0])
t_u = torch.tensor([35.7, 59.9, 58.2, 81.9, 54.3, 48.9, 30.9, 21.8, 48.4, 60.4, 68.4])
t_un = 0.1 * t_u

In [0]:
def model(t_u, w, b):
    return w * t_u + b

In [0]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [6]:
dir(optim)

['ASGD',
 'Adadelta',
 'Adagrad',
 'Adam',
 'AdamW',
 'Adamax',
 'LBFGS',
 'Optimizer',
 'RMSprop',
 'Rprop',
 'SGD',
 'SparseAdam',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'lr_scheduler']

In [0]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
learning_rate = 1e-5
optimizer = optim.SGD([params], lr=learning_rate)

In [8]:
t_p = model(t_u, *params)
loss = loss_fn(t_p, t_c)
loss.backward()
optimizer.step()
params

tensor([ 9.5336e-01, -8.4964e-04], requires_grad=True)

In [9]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)
t_p = model(t_un, *params)
loss = loss_fn(t_p, t_c)
optimizer.zero_grad() # <1>
loss.backward()
optimizer.step()

params

tensor([1.6368, 0.0811], requires_grad=True)

In [0]:
def training_loop(n_epochs, optimizer, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        t_p = model(t_u, *params) 
        loss = loss_fn(t_p, t_c)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 500 == 0:
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
            
    return params

In [11]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate) # <1>

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    params = params, # <1> 
    t_u = t_un,
    t_c = t_c)

Epoch 500, Loss 65.751572
Epoch 1000, Loss 62.102219
Epoch 1500, Loss 61.492111
Epoch 2000, Loss 61.390114
Epoch 2500, Loss 61.373058
Epoch 3000, Loss 61.370205
Epoch 3500, Loss 61.369717
Epoch 4000, Loss 61.369644
Epoch 4500, Loss 61.369640
Epoch 5000, Loss 61.369629


tensor([  5.0048, -16.6519], requires_grad=True)

In [12]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)
shuffled_indices = torch.randperm(n_samples)
train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]
train_indices, val_indices

(tensor([10,  8,  1,  4,  0,  9,  5,  7,  3]), tensor([6, 2]))

In [0]:
train_t_u = t_u[train_indices]
train_t_c = t_c[train_indices]
val_t_u = t_u[val_indices]
val_t_c = t_c[val_indices]
train_t_un = 0.1 * train_t_u
val_t_un = 0.1 * val_t_u

In [0]:
def training_loop(n_epochs, optimizer, params, train_t_u, val_t_u, train_t_c, val_t_c):
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_t_u, *params) # <1>
        train_loss = loss_fn(train_t_p, train_t_c)
                             
        val_t_p = model(val_t_u, *params) # <1>
        val_loss = loss_fn(val_t_p, val_t_c)
        
        optimizer.zero_grad()
        train_loss.backward() # <2>
        optimizer.step()
        if epoch <= 3 or epoch % 500 == 0:
            print('Epoch {}, Training loss {}, Validation loss {}'.format(
                epoch, float(train_loss), float(val_loss)))
            
    return params

In [15]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs = 3000, 
    optimizer = optimizer,
    params = params,
    train_t_u = train_t_un,
    val_t_u = val_t_un, 
    train_t_c = train_t_c,
    val_t_c = val_t_c)

Epoch 1, Training loss 138.94479370117188, Validation loss 42.140254974365234
Epoch 2, Training loss 108.88108825683594, Validation loss 16.266021728515625
Epoch 3, Training loss 104.92286682128906, Validation loss 11.800601959228516
Epoch 500, Training loss 77.42224884033203, Validation loss 6.443467140197754
Epoch 1000, Training loss 72.55511474609375, Validation loss 12.987495422363281
Epoch 1500, Training loss 71.68663024902344, Validation loss 17.232799530029297
Epoch 2000, Training loss 71.53165435791016, Validation loss 19.29034996032715
Epoch 2500, Training loss 71.50401306152344, Validation loss 20.206697463989258
Epoch 3000, Training loss 71.49907684326172, Validation loss 20.602169036865234


tensor([  5.3140, -19.0362], requires_grad=True)